* [Expell data set to files](#expell)
* [Generate a ML DataFrame](#gen_mldf)
    * [with labels as categories names](#genml_labs)
    * [with labels as categories numbers](#genml_catnumbs)
    * [with labels as 1-hot encoded numbers](#genml_1hot)
* [Export to other frameworks](#export)
    * [to Keras](#exp_keras)
    * [to FastAI](#exp_fastai)
    * [to Pytorch](#exp_pytorch)

# Data Exporting

In [ ]:
## Workspace preparation ##
import os
from pathlib import Path
import logging
logging.basicConfig()
log = logging.getLogger()
log.setLevel(logging.INFO)
logging.getLogger('PIL').setLevel(logging.INFO)
logging.getLogger('matplotlib').setLevel(logging.INFO)

In [ ]:
import cytokinin as ck
from cytokinin.data import take_data

In [ ]:
root = Path('./../') # quite the same as str

In [ ]:
# Set an example dir for images files
MOCKS = root.joinpath('./cytokinin/cytokinin/tests/mocks/')
IMGS = MOCKS/'imgs' # this is another Path object
os.listdir(str(IMGS))

## <div id='expell'>Expell to files</div>

It saves the data set, loaded by the class, into a specific place in a specific format.  
This way you can easily build your dataset, rearranging pre-existing ones.

Now load data set into the class.

In [ ]:
c = take_data('images').store_filesnames_from_folder(IMGS/'dog')
c.filesnames.head()

Before to continue create a folder "bin" in MOCKS/, to contain the results of this tutorial.

In [ ]:
os.mkdir(MOCKS/'bin')

##### Basic expell

Expell all data set into *inthisnewfolder*, preserving their names.

In [ ]:
# BEFORE TO RUN THIS: create this folder (mocks/bin) yourself in order to see what happens

path = str(MOCKS/'bin')
inthisnewfolder = 'new_images'
namefilesas = None
c.expell_to(path, inthisnewfolder, namefilesas)

##### expell_to: namefilesas 'data'

Like the Basic expell, but naming output files with the name of the object containing them as prefix.
Remember every object of the Data class has a *name* property.

In [ ]:
c.name  # files have been named with this prefix

In [ ]:
path = str(MOCKS/'bin')
inthisnewfolder = 'new_images2'
namefilesas = 'data'
c.expell_to(path, inthisnewfolder, namefilesas)

##### expell_to: namefilesas 'folder'

It expells fles in a specific folder, naming them after the folder that will contain them.  
Example:
```python
c.expell_to('imgs/', 'class5', namefilesas='folder)
os.listdir('imgs/class5')
# ['class5_dog1.png', 'class5_dog32.png', ...]
```

In [ ]:
path = str(MOCKS/'bin')
inthisnewfolder = 'new_images3'
namefilesas = 'folder'
c.expell_to(path, inthisnewfolder, namefilesas)

## <div id ='gen_mldf'>Generate a ml df</div>

In [ ]:
# Create a Data from two roots: dogs and stones
dogs = take_data('images').store_filesnames_from_folder(IMGS.joinpath('dog'))
stones = take_data('images').store_filesnames_from_folder(IMGS.joinpath('stone'))
dands = dogs.copy().add_from_data(stones)

csv_url = MOCKS/'labels'/'dogsandstones_labes.csv'
dands.label_from_csv(csv_url, col='Y')
print(dands)

#### <div id='genml_labs'> with labels as category names</div>

In [ ]:
dands.filesnames_to_ml_df()

#### <div id='genml_catnumbs'>with labels as categories numbers</div>

In [ ]:
dands.filesnames_to_ml_df(y_as='classnum').copy()

#### <div id ='genml_1hot'>with labels as 1-hot encoded</div>

In [ ]:
dands.filesnames_to_ml_df(y_as='1hot').copy()

## <div id='export'>Export</div>

### <div id ='exp_keras'>to Keras ImageDataGenerator</div>

Data.export_to_keras()

Let's try to export a not labeled data, equal a single class data, beacuse cytokinin assumes that if you're sure to ask not labeled data it means they are all from a single class.

In [ ]:
# Create a Data from two roots: dogs and stones
dogs = take_data('images').store_filesnames_from_folder(IMGS.joinpath('dog'))
stones = take_data('images').store_filesnames_from_folder(IMGS.joinpath('stone'))
dands = dogs.copy().add_from_data(stones)
print(dands)

In [ ]:
dands.export_to_keras()

Now use well labeled files (2 classes)

In [ ]:
# Create a Data from two roots: dogs and stones
dogs = take_data('images').store_filesnames_from_folder(IMGS.joinpath('dog'))
stones = take_data('images').store_filesnames_from_folder(IMGS.joinpath('stone'))
dands = dogs.copy().add_from_data(stones)

csv_url = MOCKS/'labels'/'dogsandstones_labes.csv'
dands.label_from_csv(csv_url, col='Y')
print(dands)

In [ ]:
idg = dands.export_to_keras()
idg

it also accepts [Imagedatagenerator](https://keras.io/preprocessing/image/#imagedatagenerator-class) arguments

In [ ]:
idg_args = {
    'rescale': 1./255,
    'shear_range': 0.2,
    'zoom_range': 0.2,
    'horizontal_flip': True
}
ke_gen = dands.export_to_keras(imagedatagenerator_args=idg_args)

If you want a **complete example** on how to use it with Keras, please check the [Train a Keras model with cytokinin](./examples/Train%20a%20Keras%20model.ipynb) example!

### <div id='exp_fastai'>to FastAI</div>

Data.export_to_fastAI()

In [ ]:
# Create a Data from two roots: dogs and stones
dogs = take_data('images').store_filesnames_from_folder(IMGS.joinpath('dog'))
stones = take_data('images').store_filesnames_from_folder(IMGS.joinpath('stone'))
dands = dogs.copy().add_from_data(stones)

csv_url = MOCKS/'labels'/'dogsandstones_labes.csv'
dands.label_from_csv(csv_url, col='Y')
print(dands)

To export your dataset to FastAI is very easy

In [ ]:
fastai_idb = dands.export_to_fastAI(imagedatabunch_args=idb_args)
fastai_idb

You can pass FastAI [ImageDataBunch](https://docs.fast.ai/vision.data.html#ImageDataBunch) arguments too 

In [ ]:
from fastai.vision import ImageDataBunch,ImageList
from fastai.vision.transform import get_transforms
tfms_arg = get_transforms() # https://docs.fast.ai/vision.transform.html
idb_args = {
    'ds_tfms': tfms_arg,
    'valid_pct': 0,
    'bs': 40,
#     'label_col': None, #label column
}

fastai_idb = dands.export_to_fastAI(imagedatabunch_args=idb_args)
fastai_idb

In [ ]:
type(fastai_idb)

In [ ]:
fastai_idb.open( fastai_idb.items[1] )

nice doggo

### <div id='exp_pytorch'>to Pytorch</div>

To make the Data object ready to get into a Pytorch [dataloaders](https://pytorch.org/docs/stable/data.html) you only need to specify a Pytorch [transform](https://pytorch.org/docs/stable/torchvision/transforms.html) and cytokinin will take care about the rest.  
Let's have a look to how to do it.

In [ ]:
# Create a Data from two roots: dogs and stones
dogs = take_data('images').store_filesnames_from_folder(IMGS.joinpath('dog'))
stones = take_data('images').store_filesnames_from_folder(IMGS.joinpath('stone'))
dands = dogs.copy().add_from_data(stones)

csv_url = MOCKS/'labels'/'dogsandstones_labes.csv'
dands.label_from_csv(csv_url, col='Y')
print(dands)

Just define a simple Pytorch [transform](https://pytorch.org/docs/stable/torchvision/transforms.html)

In [ ]:
import torchvision.transforms as transforms
transformations = transforms.Compose([
    transforms.Resize((224,224),interpolation=2), # NEEDED if you want to use *batch_size*
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

Set the transforms in Data object

In [ ]:
dands.set_transforms(transformations)
print(dands.torch_tsfm)

That's all!  
Now our Data object is ready to be passed to a Pytorch dataloader

In [ ]:
from torch.utils.data import Dataset, DataLoader
train_loader = DataLoader(dands, batch_size=8, shuffle=True)
train_loader

Do you want a **complete example** on how to use it with Pytorch? Check the [Train a Pytorch network with cytokinin](./examples/Train%20a%20Pytorch%20network.ipynb) example!